# Budget Analysis

---

## Environment Setup

In [1]:
# Initial Imports
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from dotenv import load_dotenv

%matplotlib inline

In [2]:
# Helper functions

def pretty_print_response(response):
    """
    This function formats a JSON file to pretty printing it in the screen.
    """
    print(json.dumps(response, indent=4, sort_keys=True))

In [3]:
# Set environment variables from the .env file
load_dotenv('/Users/connor/Documents/env/plaid.env')

True

In [4]:
# Extract API keys from environment variables
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SBX_SECRET_KEY")
PLAID_PUBLIC_KEY = os.getenv("PLAID_PUBLIC_KEY")

---

## Plaid Access Token

In this section, you will use the `plaid-python` API to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:

1. Create a client to connect to plaid

2. Use the client to generate a public token and request the following items: ['transactions', 'income', 'assets']

3. Exchange the public token for an access token

4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [5]:
# Create the client object
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SECRET, public_key=PLAID_PUBLIC_KEY, environment='sandbox')
client.Institutions.get(2)

{'institutions': [{'country_codes': ['US'],
   'credentials': [{'label': 'Username', 'name': 'username', 'type': 'text'},
    {'label': 'Password', 'name': 'password', 'type': 'password'}],
   'has_mfa': True,
   'input_spec': 'fixed',
   'institution_id': 'ins_112060',
   'mfa': ['code', 'list', 'questions', 'selections'],
   'mfa_code_type': 'numeric',
   'name': '1st Bank (Broadus, MT) - Personal',
   'oauth': False,
   'products': ['assets',
    'auth',
    'balance',
    'transactions',
    'income',
    'identity'],
   'routing_numbers': []},
  {'country_codes': ['US'],
   'credentials': [{'label': 'Username', 'name': 'username', 'type': 'text'},
    {'label': 'Password', 'name': 'password', 'type': 'password'}],
   'has_mfa': True,
   'input_spec': 'fixed',
   'institution_id': 'ins_112062',
   'mfa': ['code', 'list', 'questions', 'selections'],
   'mfa_code_type': 'numeric',
   'name': '1st Constitution Bank - Personal',
   'oauth': False,
   'products': ['assets',
    'auth',


In [6]:
# Set the institution id
INSTITUTION_ID = "ins_109508"

### 2. Generate a public token

In [7]:
# Create the public token
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions','income','assets'])

# Print the public token
print(create_tkn_response)

{'public_token': 'public-sandbox-2bac5e51-450f-4162-8201-2c2b766a15ec', 'request_id': 'fJUyX990V9nViVI'}


### 3. Exchange the public token for an access token

In [8]:
# Retrieve the access token
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])

# Print the access token
print(exchange_response)

{'access_token': 'access-sandbox-d4ee9999-6498-44f2-8caa-7c09e03dee8f', 'item_id': '8XrgV67mayIQy9RMXXGxHq8JXgnR9nUwgvPNx', 'request_id': '2zKaWR8eo6SV7Ks'}


In [9]:
# Store the access token in a Python variable
access_token = exchange_response['access_token']

### 4. Fetch Accounts

In [10]:
# Retrieve accounts
harold_accounts = client.Accounts.get(access_token)

# Print accounts
print(harold_accounts)

{'accounts': [{'account_id': '6DqpNXP6xySyZ6GR55QNIvK7Eno6gkfg5ezjE', 'balances': {'available': 100, 'current': 110, 'iso_currency_code': 'USD', 'limit': None, 'unofficial_currency_code': None}, 'mask': '0000', 'name': 'Plaid Checking', 'official_name': 'Plaid Gold Standard 0% Interest Checking', 'subtype': 'checking', 'type': 'depository'}, {'account_id': 'XJdxEoMGqKUKEZkPBBMyUk1grQVyZLcdaLDpw', 'balances': {'available': 200, 'current': 210, 'iso_currency_code': 'USD', 'limit': None, 'unofficial_currency_code': None}, 'mask': '1111', 'name': 'Plaid Saving', 'official_name': 'Plaid Silver Standard 0.1% Interest Saving', 'subtype': 'savings', 'type': 'depository'}, {'account_id': 'DXeJjoWqpyIaren7NNKyUQoeyVaJGXSvMJZmj', 'balances': {'available': None, 'current': 1000, 'iso_currency_code': 'USD', 'limit': None, 'unofficial_currency_code': None}, 'mask': '2222', 'name': 'Plaid CD', 'official_name': 'Plaid Bronze Standard 0.2% Interest CD', 'subtype': 'cd', 'type': 'depository'}, {'account

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days

2. Print the categories for each transaction type

3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)

4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [11]:
# Set the start and end date 90 days from today
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

# Retrieve the transactions for the last 90 days
transaction_response = client.Transactions.get(access_token,start_date,end_date)

# Print the transactions
print(json.dumps(transaction_response['transactions'][:2],indent=4, sort_keys=True))

[
    {
        "account_id": "VJ3BGpQLVkUNoev6XXDEIjqk9Lw1lWuWrEDb7",
        "account_owner": null,
        "amount": 78.5,
        "authorized_date": null,
        "category": [
            "Recreation",
            "Gyms and Fitness Centers"
        ],
        "category_id": "17018000",
        "date": "2020-06-12",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "name": "Touchstone Climbing",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },
        "pe

### 2. Print the categories for each transaction

In [12]:
# Create a for-loop to print the categories for each transaction
for transactions in transaction_response['transactions']:
    if transactions['category']:
        print(json.dumps(transactions['category'], indent=4, sort_keys=True))

[
    "Recreation",
    "Gyms and Fitness Centers"
]
[
    "Travel",
    "Airlines and Aviation Services"
]
[
    "Food and Drink",
    "Restaurants",
    "Fast Food"
]
[
    "Food and Drink",
    "Restaurants",
    "Coffee Shop"
]
[
    "Food and Drink",
    "Restaurants"
]
[
    "Transfer",
    "Credit"
]
[
    "Travel",
    "Airlines and Aviation Services"
]
[
    "Travel",
    "Taxi"
]
[
    "Food and Drink",
    "Restaurants"
]
[
    "Payment"
]
[
    "Food and Drink",
    "Restaurants",
    "Fast Food"
]
[
    "Shops",
    "Sporting Goods"
]
[
    "Payment",
    "Credit Card"
]
[
    "Travel",
    "Taxi"
]
[
    "Transfer",
    "Debit"
]
[
    "Transfer",
    "Deposit"
]
[
    "Recreation",
    "Gyms and Fitness Centers"
]
[
    "Travel",
    "Airlines and Aviation Services"
]


### 3. Create a new DataFrame using the following fields from the JSON transaction data: `date`, `name`, `amount`, `category`. 

(For categories with more than one label, just use the first category label in the list)

In [13]:
# Define an empty DataFrame and set the columns as date, name, amount, and category
empty_df = {'date': [],
            'name': [],
            'amount': [],
            'category': []}
category_df = pd.DataFrame(empty_df)
category_df.head()

,date,name,amount,category


In [14]:
# Retrieve all the transactions
all_transaction = transaction_response['transactions']

In [15]:
# Populate the transactions DataFrame with the transactions data
category_df = pd.DataFrame.from_dict(all_transaction)

In [16]:
category_df.columns

Index(['account_id', 'account_owner', 'amount', 'authorized_date', 'category',
       'category_id', 'date', 'iso_currency_code', 'location', 'name',
       'payment_channel', 'payment_meta', 'pending', 'pending_transaction_id',
       'transaction_code', 'transaction_id', 'transaction_type',
       'unofficial_currency_code'],
      dtype='object')

In [17]:
category_df = category_df.drop(columns=['account_id', 'account_owner', 'authorized_date',
       'category_id','iso_currency_code', 'location',
       'payment_channel', 'payment_meta', 'pending', 'pending_transaction_id',
       'transaction_code', 'transaction_id', 'transaction_type',
       'unofficial_currency_code'])

In [18]:
category_df = category_df[['date','name','amount','category']]

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [19]:
# Display data types
category_df.dtypes

date         object
name         object
amount      float64
category     object
dtype: object

In [20]:
# Make any required data type transformation
category_df['date'] = pd.DatetimeIndex(category_df['date'])

In [21]:
# Set the date column as index
category_df.set_index(category_df['date'])

# Display sample data
category_df

,date,name,amount,category
0,2020-06-12,Touchstone Climbing,78.50,"[Recreation, Gyms and Fitness Centers]"
1,2020-06-12,United Airlines,-500.00,"[Travel, Airlines and Aviation Services]"
2,2020-06-11,McDonald's,12.00,"[Food and Drink, Restaurants, Fast Food]"
3,2020-06-11,Starbucks,4.33,"[Food and Drink, Restaurants, Coffee Shop]"
4,2020-06-10,SparkFun,89.40,"[Food and Drink, Restaurants]"
5,2020-06-09,INTRST PYMNT,-4.22,"[Transfer, Credit]"
6,2020-05-30,United Airlines,500.00,"[Travel, Airlines and Aviation Services]"
7,2020-05-28,Uber,6.33,"[Travel, Taxi]"
8,2020-05-25,Tectra Inc,500.00,"[Food and Drink, Restaurants]"
9,2020-05-24,AUTOMATIC PAYMENT - THANK,2078.50,[Payment]


---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:

1. Determine the previous year's gross income and print the results

2. Determine the current monthly income and print the results

3. Determine the projected yearly income and print the results

In [24]:
# Fetch de income data
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-365))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

In [25]:
income_response = client.Income.get(access_token)

# Print the income data
print(json.dumps(income_response['income'],indent=4, sort_keys=True))

{
    "income_streams": [
        {
            "confidence": 0.99,
            "days": 720,
            "monthly_income": 500,
            "name": "UNITED AIRLINES"
        }
    ],
    "last_year_income": 6500,
    "last_year_income_before_tax": 7893,
    "max_number_of_overlapping_income_streams": 1,
    "number_of_income_streams": 1,
    "projected_yearly_income": 6085,
    "projected_yearly_income_before_tax": 7389
}


In [26]:
# Determine the previous year's gross income and print the results
print(income_response["income"]["last_year_income_before_tax"])

7893


In [27]:
# Determine the current monthly income and print the results
income_response["income"]["income_streams"][0]['monthly_income']

500

In [28]:
# Determine the projected yearly income and print the results
print(income_response["income"]["projected_yearly_income"])

6085


---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)

2. Generate a bar chart with the number of transactions for each category

3. Calulate the expenses per month

4. Plot the total expenses per month

### Calculate the expenses per category

In [29]:
# Compute the total expenses per category
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-365))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

In [30]:
# Retrieve the transactions for the last 365 days
transaction_response_365 = client.Transactions.get(access_token,start_date,end_date)

In [39]:
# Print the transactions
all_transactions_365 = transaction_response_365['transactions']

In [40]:
category_365 = pd.DataFrame.from_dict(all_transactions_365)

In [45]:
category_365 = category_365.drop(columns=['account_id', 'account_owner', 'authorized_date',
       'category_id','iso_currency_code', 'location',
       'payment_channel', 'payment_meta', 'pending', 'pending_transaction_id',
       'transaction_code', 'transaction_id', 'transaction_type',
       'unofficial_currency_code', 'date', 'name'])

In [82]:
category_365['category'].value_counts()

[Food and Drink, Restaurants, Fast Food]      13
[Travel, Airlines and Aviation Services]      13
[Food and Drink, Restaurants]                 12
[Travel, Taxi]                                12
[Recreation, Gyms and Fitness Centers]         7
[Food and Drink, Restaurants, Coffee Shop]     7
[Payment, Credit Card]                         6
[Transfer, Deposit]                            6
[Shops, Sporting Goods]                        6
[Transfer, Debit]                              6
[Transfer, Credit]                             6
[Payment]                                      6
Name: category, dtype: int64

In [85]:
# Display the expenses per category

KeyError: 'Food and Drink, Restaurants, Fast Food'

In [ ]:
# Create a spending categories pie chart.


### Calculate the expenses per month

In [ ]:
# Create a DataFrame with the total expenses

# Display sample data


In [ ]:
# Create a spending per month bar chart
